In [4]:
import cx_Oracle as cx
cn=cx.connect('scott/tiger@127.0.0.1/orcl')
cur=cn.cursor()
print(cur)


<cx_Oracle.Cursor on <cx_Oracle.Connection to scott@127.0.0.1/orcl>>


In [5]:
import datetime

def invoice_header():
    in_id=int(input("enter the invoice Id: "))
    InvDt=datetime.datetime.now()
    header_para1=[in_id,InvDt]
    
    #print(header_para)
    header_ins1='''
    insert into tbl_sales_invoice_header(InvId,InvDt) values(:InvId,:InvDt)
    '''
    cur.execute(header_ins1,header_para1)
    total_price=invoice_details(in_id)
    req=input("Press Y/y to enter the header discount")
    if req=='y' or req=='Y':
        disc=float(input("enter the discount"))
    else:
        disc=0.0
    header_discount=total_price-disc
    header_para2=[total_price,header_discount,in_id]
    header_ins2='''
    update tbl_sales_invoice_header set InvTotalPrice=:InvTotalPrice, InvHeaderDisc=:InvHeaderDisc
    where InvId=:InvId
    '''
    cur.execute(header_ins2,header_para2)
    cn.commit()
    header_rec=cur.execute('select * from tbl_sales_invoice_header')
    for r in header_rec:
        print(r)
    
def invoice_details(in_id):
    #disc=5
    InvId=in_id
    LineNo=int(input("enter the line no. "))
    P_Id=int(input("enter the product Id"))
    ans=cur.execute('select ProdSellPrice from tbl_mst_product where ProdId='+str(P_Id))
    price=ans.fetchone()
    #print(price)
    qty=int(input("enter the quantity"))
    sell_price=price[0]*qty
    discount=sell_price-0.05*sell_price
    #total_price=discount
    inv_para=[InvId,LineNo,P_Id,qty,sell_price,discount]
    inv_ins='''
    insert into tbl_sales_invoice_details(InvId,LineNo,ProdId,Quantity,SellPrice,Discount)
    values(:InvId,:LineNo,:ProdId,:Quantity,:SellPrice,:Discount)
    '''
    cur.execute(inv_ins,inv_para)
    inv_rec=cur.execute('select * from tbl_sales_invoice_details order by InvId ASC')
    for r in inv_rec:
        print(r)
    ans1=cur.execute('select sum(Discount) from tbl_sales_invoice_details where InvId='+str(in_id))
    total_price=ans1.fetchone()
    return(total_price[0])
    

    
invoice_header()

enter the invoice Id: 9
enter the line no. 1
enter the product Id2
enter the quantity3
(1, 1, 4, 3, 2400.0, 2280.0)
(9, 1, 2, 3, 2100.0, 1995.0)
Press Y/y to enter the header discountn
(1, 2280.0, 2280.0, datetime.datetime(2020, 5, 4, 1, 31, 31))
(9, 1995.0, 1995.0, datetime.datetime(2020, 5, 4, 1, 33, 33))
